# Hagen-Poiseuille flow


## Description

Simulate transition from uniform plug inlet to fully developed parabolic profile in laminar pipe flow.

### Entrance length (99% developed)

L_e / D ≈ 0.06 × Re_D (theoretical approximation for Re < 2300)
### Recommended domain length
L ≥ 1.5–2.5 × L_e (ideally L/D ≈ 12–20)

### Typical test case (recommended starting point)

- Radius r = 0.5 → D = 1.0
- Re_D = 100 → L_e ≈ 6.0
- Tube length L = 15.0 (L/D = 15)
- Inlet: uniform plug velocity U_avg = U_inlet = 0.1 (along z)
- Developed centerline velocity U_max = 2 × U_avg = 0.2
- Fluid: ρ = 1.0, μ = (ρ U_avg D) / Re_D = 0.001
- Pressure gradient G = 8 μ U_avg / r² ≈ 0.0032 (for reference)

## Mesh recommendations

- Radial resolution: ≥ 10–12 points from center to wall
- Axial spacing: Δz ≈ 0.05–0.15
- cube_to_tube(r=0.5, refinements=2–3, height=L)
- Use cdist=1e-10 for merging

## Boundary conditions summary

- Walls: no-slip (u = 0 on side_boundary)
- Inlet: new vertex layers with fixed plug velocity U_z = U_avg
- Outlet: free outflow (remove exiting vertices, fixed P=0 or do-nothing)

## Validation targets

- Radial velocity profiles u_z(r) at z = 0, L_e/2, L_e, 1.5 L_e, 2 L_e
- Centerline velocity growth vs z
- Mass conservation (total ∑ m_v should stay nearly constant)


### Analytical equilibrium initial conditions and tests

Key assumptions (tunable via parameters):

- Tube/channel along z-axis (3D) or x-axis (2D planar).
- No-slip walls (velocity = 0 at boundary).
- 3D: axisymmetric Hagen-Poiseuille, radial coordinate r = sqrt(x² + y²), max centerline velocity U_max.
- 2D: parabolic u_x(y) between plates at y = y_lb and y = y_ub.
- Pressure stored as vector v.P (diagonal gradient potential, per your code).
- Velocity stored as v.u (vector).
- Use your dudt, dP, du operators (they already support dim=3 via e_star/v_star).

Summary of parameters, ICs and BCs:

In [8]:
# General imports
import numpy as np
from scipy.spatial import Delaunay  # if needed for mesh checks

# Import parameters from _params.py
from src._params import r, D, L, Re_D, U_avg, U_max, rho, mu, L_e_approx, G, refinements, cdist, inlet_layer_thickness, outlet_buffer, add_inlet_every, CFL_target, print_params
print_params()

# Import analytical solutions and IC/ test functions
from src._analytical_equil import (
    poiseuille_analytical_2d,
    poiseuille_analytical_3d,
    P_gradient_analytical,
    set_equilibrium_IC_2d,
    set_equilibrium_IC_3d,
    test_analytical_equilibrium_2d,
    test_analytical_equilibrium_3d
)

=== Hagen–Poiseuille Developing Flow Configuration ===
Radius r          = 0.5 m
Diameter D        = 1.0 m
Tube length L     = 15.0 m  (L/D = 15.0)
Re_D              = 100
Entrance length   ≈ 6.00 m  (L_e/D ≈ 6.0)
Inlet U_avg       = 0.1 m/s
Developed U_max   = 0.2 m/s
Density ρ         = 1.0 kg/m³
Viscosity μ       = 0.00100 Pa·s
Pressure gradient G = 0.00320 Pa/m


Usage notes:

- Call set_equilibrium_IC_2d(HC, ...) or _3d after mesh creation (cube_to_tube or your incomp_poiseuille_2d).
- Adjust G (pressure gradient) or U_max/R to match your physical scaling.
- Run the test_* functions immediately after setting ICs; dudt should be machine-zero for exact equilibrium (up to discretization error in du/dP).
- If v.P vector length needs explicit 3-component handling, extend dP/du calls accordingly (your dim=3 path already exists).

In [9]:
# Library specific imports
from ddgclib.dynamic_integrators._integrators_dynamic import dudt